In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
import psycopg2
from tqdm import tqdm
import json
from psycopg2 import sql
database = {
    "database": "postgres",
    "user": "postgres",
    "password": "password",
    "host": "192.168.1.16",
    "port": "5432"
}

def connect_to_db():
    return psycopg2.connect(
        dbname=database["database"],
        user=database["user"],
        password=database["password"],
        host=database["host"],
        port=database["port"]
    )

/Users/tefannastasa/Library/Caches/pypoetry/virtualenvs/fii-advrn-project-h28zvCYG-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
from dataclasses import dataclass
import base64

def encode_base64(value):
    """
    Encodes a string into a Base64-encoded string using UTF-8.

    Args:
        value (str): The input string to encode.

    Returns:
        str: The Base64-encoded string, or None if the input is None.
    """
    if value is None:
        return None
    return base64.b64encode(value.encode('utf-16')).decode('ascii')

def decode_base64(value):
    """
    Decodes a Base64-encoded string back to its original UTF-8 string.

    Args:
        value (str): The Base64-encoded string to decode.

    Returns:
        str: The original decoded string, or None if the input is None.
    """
    if value is None:
        return None
    # Normalize padding
    value = value.strip()
    value += "=" * ((4 - len(value) % 4) % 4)
    return base64.b64decode(value.encode('ascii')).decode('utf-16')

@dataclass
class Subject:
    code: str
    name: str
    classification_name: str
    alternative_names: list[str]
    related: list[str]

In [47]:
import json, urllib, itertools
with open("../llms4subjects/shared-task-datasets/GND/dataset/GND-Subjects-tib-core.json", "r") as gnd_subjects_file:
    gnd_subjects = json.load(gnd_subjects_file)

subjects = [
    Subject(
        code=subject["Code"],
        name=urllib.parse.quote(subject["Name"]),
        classification_name=urllib.parse.quote(subject["Classification Name"]),
        alternative_names=[
            urllib.parse.quote(alternative_name)
            for alternative_name in subject["Alternate Name"]
        ],
        related=[
            urllib.parse.quote(related)
            for related in subject["Related Subjects"]
        ]
    )
    for subject in gnd_subjects
]

conn = connect_to_db()
curs = conn.cursor()
curs.execute("SET search_path TO ag_catalog;")
batch_size = 1000
total = len(subjects) // batch_size
for subject in tqdm(itertools.batched(subjects, batch_size), total=total):

    data = [(s.code, s.name) for s in subject]

    curs.executemany(
        f"""
        SELECT * FROM cypher('gnd', $$
            MATCH (d:Subject {{code: %s}})
            SET d.name = %s
            RETURN d
        $$) AS (result agtype)
        """, data
    )

conn.commit()
conn.close()


80it [08:53,  6.67s/it]                        
